In [1]:
import pandas as pd

n_campi = 'CIGR'

levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CIGR.xlsx')
suap = pd.read_excel('consolidacao/suap/2023-08-03_16-16_CIGR_relatorio_almox.xlsx')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  172 non-null    datetime64[ns]
 1   Endereço de e-mail                    172 non-null    object        
 2   Retificação                           10 non-null     object        
 3   Código                                172 non-null    object        
 4   Nome                                  172 non-null    object        
 5   Quantidade                            172 non-null    int64         
 6   Campus                                172 non-null    object        
 7   Localização                           59 non-null     object        
 8   Valor unitário                        172 non-null    float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,9,11 - MATERIAL QUIMICO
1,30.14.177,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",UN,4,14 - MATERIAL EDUCATIVO E ESPORTIVO
2,30.14.24,BOMBA DE ENCHER BOLA,UN,3,14 - MATERIAL EDUCATIVO E ESPORTIVO
3,30.14.278,COLETE ESPORTIVO POLIESTER.,UN,100,14 - MATERIAL EDUCATIVO E ESPORTIVO
4,30.14.283,"COLETE ESPORTIVO, MATERIAL POLIÉSTER, NUMERADO...",UN,24,14 - MATERIAL EDUCATIVO E ESPORTIVO
...,...,...,...,...,...
164,30.36.532,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,Conjunto,1,36 - MATERIAL HOSPITALAR
165,30.42.280,ARCO SERRA 10 E 12 POL. TIPO REGULÁVEL.,UN,1,42 - FERRAMENTAS
166,30.42.877,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",JOGO,1,42 - FERRAMENTAS
167,30.44.61,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,30.16.2168,"CANETA ESFEROGRÁFICA BIC, COR: PRETA",57,claudeandela.cavalcanti@reitoria.ifpe.edu.br
1,30.16.713,"CANETA ESFEROGRÁFICA, MATERIAL PLÁSTICO, QUANT...",54,claudeandela.cavalcanti@reitoria.ifpe.edu.br
2,30.16.572,PONTA REFIL MARCADOR RECARREGÁVEL PARA QUADRO ...,31,claudeandela.cavalcanti@reitoria.ifpe.edu.br
3,30.16.169,CLIPE DE METAL 2/0,66,claudeandela.cavalcanti@reitoria.ifpe.edu.br
4,30.16.684,"CLIPE II - TAMANHO 8/0, CAIXA COM 25 UN",107,claudeandela.cavalcanti@reitoria.ifpe.edu.br
...,...,...,...,...
167,30.16.2416,"PAPEL COUCHÊ, 180 G, A4, PACOTE C. 50 FOLHAS.",6,edivonaldo.araujo@igarassu.ifpe.edu.br
168,30.16.919,"SACO PLÁSTICO PARA DOCUMENTOS, USO EM PASTA AZ...",9,edivonaldo.araujo@igarassu.ifpe.edu.br
169,30.16.2417,"TNT, TECIDO FIBRA SINTETICA",43,edivonaldo.araujo@igarassu.ifpe.edu.br
170,30.22.708,TAMPA DE VASO SANITARIO.,17,cmpsg@igarassu.ifpe.edu.br


In [5]:
consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.847,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",UN,17,07 - GENEROS DE ALIMENTACAO,CAF,17,cmpsg@jaboatao.ifpe.edu.br,both,0
1,30.16.1026,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",UN,5,16 - MATERIAL DE EXPEDIENTE,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",5,claudeandela.cavalcanti@reitoria.ifpe.edu.br,both,0
2,30.16.1093,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",UN,4,16 - MATERIAL DE EXPEDIENTE,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",4,cmpsg@jaboatao.ifpe.edu.br,both,0
3,30.16.1096,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",CAIXA,4,16 - MATERIAL DE EXPEDIENTE,"COLCHETE (BAILARINA), N",4,edmar.filho@reitoria.ifpe.edu.br,both,0
4,30.16.1144,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",UN,1364,16 - MATERIAL DE EXPEDIENTE,"PASTA SUSPENSA, MATERIAL CART",1364,cmpsg@jaboatao.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...,...
222,30.42.1160,"TORNEIRA, Material corpo metal cromado, tipo l...",UN,29,42 - FERRAMENTAS,"TORNEIRA, Material corpo metal cromado, tipo l...",29,ceof@jaboatao.ifpe.edu.br,both,0
223,30.44.41,Círculo ou seta indicativo de H2O. Cor interna...,UN,5,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de H2O. Cor interna...,5,ceof@jaboatao.ifpe.edu.br,both,0
224,30.44.42,Círculo ou seta indicativo de CO2. Cor interna...,UN,16,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de CO2. Cor interna...,16,ceof@jaboatao.ifpe.edu.br,both,0
225,30.44.43,Círculo ou seta indicativo de PQS. Cor interna...,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de PQS. Cor interna...,2,ceof@jaboatao.ifpe.edu.br,both,0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.847,07 - GENEROS DE ALIMENTACAO,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",17,17,0,adequado
1,30.16.1026,16 - MATERIAL DE EXPEDIENTE,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",5,5,0,adequado
2,30.16.1093,16 - MATERIAL DE EXPEDIENTE,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",4,4,0,adequado
3,30.16.1096,16 - MATERIAL DE EXPEDIENTE,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",4,4,0,adequado
4,30.16.1144,16 - MATERIAL DE EXPEDIENTE,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",1364,1364,0,adequado
...,...,...,...,...,...,...,...
222,30.42.1160,42 - FERRAMENTAS,"TORNEIRA, Material corpo metal cromado, tipo l...",29,29,0,adequado
223,30.44.41,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de H2O. Cor interna...,5,5,0,adequado
224,30.44.42,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de CO2. Cor interna...,16,16,0,adequado
225,30.44.43,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,Círculo ou seta indicativo de PQS. Cor interna...,2,2,0,adequado
